# Parallel Magnetic Resonance imaging (MRI)

Standard MRI is described by the Fourier transform $\mathcal{F}$ as forward operator (here in two dimensions). 
To accelerate data acquisition, parallel MRI uses simultaneous measurements by $N$ receiver coils. This allows undersampling 
of the Fourier domain leading to speed-ups. Parallel MRI is described by the forward operator
$$F\left(\begin{array}{c}\rho \\ c_1\\ \vdots \\ c_N\end{array}\right) 
= \left(\begin{array}{c}M\cdot\mathcal{F}(c_1 \cdot \rho)\\ \vdots \\
M\cdot\mathcal{F}(c_N \cdot \rho)\end{array}\right).$$
Here $\rho$ describes the hydrogen density and is the main quantity of interest. To take into account effects such as motion artifacts, $\rho$ has to be modelled as a complex-valued function. $c_1,\dots, c_N$ describe complex-valued coil profiles, which may be assumed to be smooth. As they depend on the sample $\rho$, they must be reconstructed together with $\rho$. $M$ is a 0-1-mask describing the undersampling pattern.  

## Import of auxiliary packages and defining logging

We will `numpy` arrays to model the complex functions and rely on `matplotlib` to plot. `loadmat` from `scipy.io` is required to import the data.

In [ ]:
import logging

import matplotlib.pyplot as plt
import matplotlib as mplib
from matplotlib.colors import hsv_to_rgb
import numpy as np
from scipy.io import loadmat

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-40s :: %(message)s'
)

## Complex to rgb conversion

Converts array of complex numbers into array of RGB color values for plotting. The hue corresponds to the argument.
The brighntess corresponds to the absolute value.  

`Parameters`
>z : `numpy.ndarray`
>array of complex numbers

`Returns`
> `numpy.ndarray`
> Array that contains three values for each value in z containing the RGB representation of this value.

In [ ]:
def complex_to_rgb(z):
    HSV = np.dstack( (np.mod(np.angle(z)/(2.*np.pi),1), 1.0*np.ones(z.shape), np.abs(z)/np.max((np.abs(z[:]))), ))
    return hsv_to_rgb(HSV)

## Load data from file and estimate sampling pattern

First this code will import the data from the data file and normalizes the data.

Using the data we can estimate the sampling pattern using one of our method. This estimation relies on the approach that if some measurement point is zero in all coil profiles it is assumed to be outside of the sampling pattern. This method has a very low probability of failing, especially non-integer data.

In [ ]:
from examples.mri.mri import  estimate_sampling_pattern

data = loadmat('data/ksp3x2.mat')['Y']

data = np.transpose(data,(2,0,1))*(100/np.linalg.norm(data))

# normalize and transpose data 
nrcoils, n1, n2 = data.shape

mask = estimate_sampling_pattern(data)
plt.imshow(mask.T); plt.title('Undersampling pattern of data')

## Set up forward operator

Using the from the data extracted dimensions define the grid for the functions of the densities.

The full MRI operator is then constructed on the grid with the number of coils extracted from the data. This is done by the auxiliary method `parallel_mri` that constructs the operator as a composition of the implemented coil multiplier and a Fourier transform:

```
def parallel_mri(grid, ncoils, centered=False):
    cmult = CoilMult(grid, ncoils)
    ft = FourierTransform(cmult.codomain, axes=range(1, cmult.codomain.ndim), centered=centered)
    return ft * cmult
```

Using the sampling mask extracted from the data define the sampling operator as a point-wise multiplication with the mask. 

As a last step we define a sobolev smoothing that we be used before the full MRI operator. Finally, the parallel MRI operator can be defined as the simple composition of the before constructed operators.  

In [ ]:
from examples.mri.mri import  parallel_mri, sobolev_smoother
from regpy.operators import PtwMultiplication
from regpy.vecsps import UniformGridFcts

grid = UniformGridFcts((-1, 1, n1), (-1, 1, n2), dtype=complex)

full_mri_op = parallel_mri(
    grid=grid, 
    ncoils=nrcoils,
    centered=True
)

sampling = PtwMultiplication(full_mri_op.codomain,(1.+0j)* mask)

sobolev_index = 32
smoother = sobolev_smoother(full_mri_op.domain, sobolev_index, factor=220.)

parallel_mri_op = sampling * full_mri_op * smoother

## The regularization process
### Set up initial guess

We use constant density and zero coil profiles as initial guess. Since the domain is a direct sum we can of the density and the coil profiles we can use a splitting to extract the density and separately set this to be constant one. 

In [ ]:
init = parallel_mri_op.domain.zeros()
init_density, _ = parallel_mri_op.domain.split(init)
init_density[...] = 1

### Set up regularization method

Before we can define a solver we define a regularization setting using $L^2$ penalty and data-fidelity. 

As this problem is a non-linear problem we rely on an iteratively regularized Gauss-Newton solver to regularize this problem.  

Since the solver should not require many iterations we can use as a stopping rule an iteration count of 5.

In [ ]:
from regpy.solvers import RegularizationSetting
from regpy.hilbert import L2
from regpy.solvers.nonlinear.irgnm import IrgnmCG
from regpy.stoprules import CountIterations

setting = RegularizationSetting(op=parallel_mri_op, penalty=L2, data_fid=L2)

solver = IrgnmCG(
    setting=setting,
    data=data,
    regpar=1,
    regpar_step=1/3.,
    init=init
)

stoprule = CountIterations(max_iterations=5) 

### Run solver and plot iterates

To be able to plot the iterations steps we use the `while_` method from the solvers that yields the iterates in each step.

In the plots you can see on the left side the density function represented as a greyscale image and on the right you see the complex coil sensitivities plotted in a rgb style mapping the complex to rgb using the above defined `complex_to_rgb` method. 

In [ ]:
for reco, reco_data in solver.while_(stoprule):
    rho, coils = smoother.codomain.split(smoother(reco))
    #rho, coils = normalize(rho,coils)

    fig = plt.figure(figsize = (15,9))

    gs = fig.add_gridspec(3,7)
    axs = [fig.add_subplot(gs[0:3, 0:3])]
    axs[0].imshow(np.abs(rho),cmap=mplib.colormaps['Greys_r'],origin='lower')
    axs[0].xaxis.set_ticklabels([])
    axs[0].yaxis.set_ticklabels([])
    for j in range(3):
        for k in range(3,7):
            axs.append(fig.add_subplot(gs[j,k]))
            axs[-1].xaxis.set_ticklabels([])
            axs[-1].yaxis.set_ticklabels([])
    for j in range(nrcoils):
        axs[1+j].imshow(complex_to_rgb(coils[j,:,:]),origin='lower')
    plt.show()